In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import matplotlib.animation as animation

In [2]:
data  = pd.read_csv("../results/csv/final_automated_wether_data1.csv")

In [3]:
data.head()

Id        Date         Time  Longitude_x  Latitude_x  Tempreture  Humidity  \
0   1  2023-11-23  18:47:21.38   143.251973  -35.083201       29.67        28   
1   2  2023-11-23  18:47:21.38   143.252973  -35.083201       29.67        28   
2   3  2023-11-23  18:47:21.38   143.253973  -35.083201       29.67        28   
3   4  2023-11-23  18:47:21.38   143.254973  -35.083201       29.67        28   
4   5  2023-11-23  18:47:22.64   143.255973  -35.083201       29.64        28   

   Wind speed  Wether ID Wether ID group  ... Weather condition prob  \
0        5.45        500            Rain  ...               0.166667   
1        5.45        500            Rain  ...               0.166667   
2        5.45        500            Rain  ...               0.166667   
3        5.45        500            Rain  ...               0.166667   
4        5.43        500            Rain  ...               0.166667   

  hour prob tempreture prob  humidity prob  wind prob  Weather Prob  \
0       1.0             1.0       0.529518        1.0      0.088253   
1       1.0             1.0       0.529518        1.0      0.088253   
2       1.0             1.0       0.529518        1.0      0.088253   
3       1.0             1.0       0.529518        1.0      0.088253   
4       1.0             1.0       0.529518        1.0      0.088253   

   Longitude_y  Latitude_y  Spatial Prob  Final Prob  
0   143.251973  -35.083201      0.036364    0.003209  
1   143.252973  -35.083201      0.070245    0.006199  
2   143.253973  -35.083201      0.121436    0.010717  
3   143.254973  -35.083201      0.177571    0.015671  
4   143.255973  -35.083201      0.215976    0.019061  

[5 rows x 24 columns]

In [10]:
lat = data["Latitude_x"].values
long = data["Longitude_x"].values

lat2 = data["Latitude_y"].values
long2 = data["Longitude_y"].values

In [14]:
(lat == lat2).all()

True

In [22]:
(long == long2).all()

True

In [73]:
plt.figure(figsize=(12,12))
plt.scatter(long,lat)
#plt.scatter(long2,lat2)

plt.scatter(long[65],lat[65], c='r' )
plt.show()

### This code shows how the grid points are created 

In [74]:

%matplotlib tk
fig, ax = plt.subplots()
t = long

z = lat

scat = ax.scatter(t[0], z[0], c="b", s=5)

ax.set(xlim=[min(long), max(long)], ylim=[min(lat),max(lat)])


def update(frame):
    # for each frame, update the data stored on each artist.
    x = t[:frame]
    y = z[:frame]
    # update the scatter plot:
    data = np.stack([x, y]).T
    scat.set_offsets(data)
 
    return (scat)


ani = animation.FuncAnimation(fig=fig, func=update, frames=len(lat), interval=1)
plt.show()

In [91]:
import math 

math.trunc(5.99999999)

5

### Old code for the weather data download function (before speed up adding)

In [ ]:
"""def download_weather_data_raw(latitudes,longitudes,cols):
    
    # this function extract the weather data from api when provide the lat and long arrays (each raw of latitude)
    # here cols means number of points in a raw
    # create a data frame
    grid_point_wether_data = pd.DataFrame(columns=["Time","Longitude", "Latitude","Tempreture", "Humidity","Wind speed","Wether ID", "Wether ID group", "Wether ID description", "Sunrise", "Sunset"])
    srt_time  = datetime.now()
    piangil_timezone = pytz.timezone('Australia/Sydney')

    for i in range(int(cols/4)): # contralls the amount of the data

        srt_time_point  = datetime.now()
        #get the lat long coordinates
        lat = latitudes[4*i] 
        long = longitudes[4*i]

        #API url
        url = "https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid=8ee842d65cf08ec205365865e3d53348&units=metric".format(lat,long)

        piangil_time = datetime.now(piangil_timezone) #get time in Australia for data set

        #get data form API as json data
        res = requests.get(url)
        data = res.json()

        # create the data list that we want from the json data 
        data_vec = [piangil_time,long, lat, data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
        data_vec_1 = [piangil_time,longitudes[4*i+1], latitudes[4*i+1], data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
        data_vec_2 = [piangil_time,longitudes[4*i+2], latitudes[4*i+2], data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
        data_vec_3 = [piangil_time,longitudes[4*i+3], latitudes[4*i+3], data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]


        #update the data frame
        grid_point_wether_data.loc[4*i] = data_vec
        grid_point_wether_data.loc[4*i+1] = data_vec_1
        grid_point_wether_data.loc[4*i+2] = data_vec_2
        grid_point_wether_data.loc[4*i+3] = data_vec_3

        # if the longitudes arr length (or raw length of the map points) can not divide by 4 then remaining point in the columns should be filled previous values
        if(i%((int(cols/4))-1)==0) and (cols%4 !=0) and (i!=0):
            num = cols%4
            for j in range(num):
                data_vec_j = [piangil_time,longitudes[4*i+3+(j+1)], latitudes[4*i+3+(j+1)], data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
                grid_point_wether_data.loc[4*i+3+(j+1)] = data_vec_j
                print(f"this is done when i is equals to {i}")


        time.sleep(0.1)
        end_time_point  = datetime.now()
        print(f"step {i+1} is completed! and taken {end_time_point-srt_time_point} time to complete")


    end_time = datetime.now()
    total_execution_time = end_time-srt_time
    print(f"the programe take: {total_execution_time} to complete")
    
          
    return grid_point_wether_data




def download_weather_data(latitudes,longitudes,cols,raws):
    
    grid_point_wether_data = pd.DataFrame(columns=["Time","Longitude", "Latitude","Tempreture", "Humidity","Wind speed","Wether ID", "Wether ID group", "Wether ID description", "Sunrise", "Sunset"])
    
    for i in range(raws):
        
        # selecting each raw of latitude and longitude arrays
        lat_arr = latitudes[i*cols:(i+1)*cols]
        long_arr = longitudes[i*cols:(i+1)*cols] 
        
        # get weather data for each raw of latitudes and longitudes
        first_batch_data = download_weather_data_raw(lat_arr,long_arr,cols)
        
        # combine the pandas dataframe with previoues one
        grid_point_wether_data = pd.concat([grid_point_wether_data,first_batch_data], axis=0, ignore_index=True)
        print(f"complete the {i+1} raw data download")
        print("==================")
        print("==================")
    
    # set the Id column and charge the raw order
    grid_point_wether_data["Id"] = [j+1 for j in range(cols*raws)]
    grid_point_wether_data = grid_point_wether_data[["Id","Time","Longitude", "Latitude","Tempreture", "Humidity","Wind speed","Wether ID", "Wether ID group", "Wether ID description", "Sunrise", "Sunset"]]
    
    return grid_point_wether_data"""